In [1]:
import os
import torch
import torchvision

from pipeline.train import Trainer
from model_architecture.darknet import YOLO
from pipeline.deeplontardataset import visualize_dataset
from pipeline.deeplontardataset import DeepLontarDataset

In [3]:
data_root_dir = 'C:\\Users\\rudyh\\Documents\\Python\\datasets\\DeepLontar-Dataset'
image_dir = os.path.join(data_root_dir, 'images')
label_dir = os.path.join(data_root_dir, 'labels')

input_shape = (704, 128)
dataset = DeepLontarDataset(image_dir=image_dir, label_dir=label_dir)
train_set, test_set = dataset.split_dataset(split_ratio=0.8)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False, num_workers=4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

"""
An anchor box is a bounding box with various scales and aspect ratios
centered on each pixel. The anchor box samples a large number of regions 
in the input image and determines whether these regions contain objects of 
interest and adjusts the region boundaries to predict with better accuracy 
the ground truth bounding box of the object. 
Other parameters also come from below paper.
Paper: https://ieeexplore.ieee.org/document/9694598
"""
anchors = [[37, 49], [65, 3], [46, 75], [63, 58], [68, 85], [95, 73], [96, 105], [127, 303], [294, 137]]

momentum = 0.949
num_classes = 55
decay = 5e-4

# Define model
batch_size = 4
learning_rate = 1e-3
num_epochs = 3

# Data Transformation
data_transform = torchvision.transforms.Compose([
	torchvision.transforms.ToTensor(),
	torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
import numpy as np
anchors = [[37, 49], [65, 3], [46, 75], [63, 58], [68, 85], [95, 73], [96, 105], [127, 303], [294, 137]]
print(np.array(anchors).reshape((-1, 2)))

[[ 37  49]
 [ 65   3]
 [ 46  75]
 [ 63  58]
 [ 68  85]
 [ 95  73]
 [ 96 105]
 [127 303]
 [294 137]]


In [5]:
[(input_shape[0] // s, input_shape[1] // s) for s in [32, 16, 8]]

[(22, 4), (44, 8), (88, 16)]